<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/multi_cls_llm_embed_llama3_2_3Bipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers tqdm datasets

In [ ]:

train_path = ('/content/drive/MyDrive/YonseiAI/rev_tag_training_samples.csv')
valid_path = ('/content/drive/MyDrive/YonseiAI/rev_tag_validation_samples.csv')

In [ ]:
from datasets import load_dataset
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

# 멀티 레이블 데이터셋 로드
sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/YonseiAI/rev_tag_training_samples.csv',
    split='train'
)

# Title과 Body를 결합하여 Body 업데이트
def update_body(example):
    updated_body = example['Title'] + ' ' + example['Body']
    example['Body'] = updated_body
    return example

# map 함수를 사용하여 모든 데이터에 update_body 함수 적용
sof_dataset_train = sof_dataset_train.map(update_body)

# # 변경된 데이터셋 출력
# print(sof_dataset_train['Title'][0])
# print(sof_dataset_train['Body'][0])
print(sof_dataset_train)

# 레이블 이름으로 구성된 리스트
labels = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

# 레이블 이름을 키로 하고 카운터를 값으로 하는 딕셔너리 생성
label_counter = {label: 0 for label in labels}

for example in sof_dataset_train:
    # 각 샘플에서 0과 1로 표현된 레이블 상태를 이용해 카운트를 업데이트합니다.
    for label in labels:
        if example[label] == 1:
            label_counter[label] += 1

print(label_counter)

In [ ]:
# UTF-8 로케일 설정을 환경 변수로 지정
import os
os.environ["LANG"] = "C.UTF-8"
os.environ["LC_ALL"] = "C.UTF-8"

# 설치 시 환경 변수 확인
!echo $LANG
!echo $LC_ALL


In [ ]:
from datasets import load_dataset

# 멀티 레이블 데이터셋 로드
sof_dataset_val = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/YonseiAI/rev_tag_validation_samples.csv',
    split='train'
)

# Title과 Body를 결합하여 Body 업데이트
def update_body(example):
    updated_body = example['Title'] + ' ' + example['Body']
    example['Body'] = updated_body
    return example

# map 함수를 사용하여 모든 데이터에 update_body 함수 적용
sof_dataset_val = sof_dataset_val.map(update_body)

# print(sof_dataset_val['Title'][0])
# print(sof_dataset_val['Body'][0])
print(sof_dataset_val)

# 레이블 이름으로 구성된 리스트
labels = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

# 레이블 이름을 키로 하고 카운터를 값으로 하는 딕셔너리 생성
label_counter = {label: 0 for label in labels}

for example in sof_dataset_val:
    # 각 샘플에서 0과 1로 표현된 레이블 상태를 이용해 카운트를 업데이트합니다.
    for label in labels:
        if example[label] == 1:
            label_counter[label] += 1

print(label_counter)

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict

sof_dataset= DatasetDict({
    "train": sof_dataset_train,
    "test": sof_dataset_val
})

# DatasetDict 출력
print(sof_dataset)


# **1. Llama3.2 3B Embedding**

In [14]:
pip install huggingface_hub


In [15]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `embedllama3b` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `embed

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from tqdm import trange
import pandas as pd

def rep_extract(task, mode, device, sents, labels, max_len, step):
    model_id = "meta-llama/Llama-3.2-3B"  # 3B 모델로 변경

    # 인증 토큰 없이 토크나이저 로드 (CLI 로그인 사용)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    config_kwargs = {
        "trust_remote_code": True,
        "cache_dir": None,
        "revision": 'main',
        "output_hidden_states": True
    }
    model_config = AutoConfig.from_pretrained(model_id, **config_kwargs)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        device_map=device,
        torch_dtype=torch.float16
    )
    model.eval()

    sents_reps = []
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding, output_hidden_states=True)
            reps_batch_5L = []
            for layer in range(-1, -6, -1):
                reps_batch_5L.append(torch.mean(batch_outputs.hidden_states[layer], axis=1))
            reps_batch_5L = torch.stack(reps_batch_5L, axis=1)

        sents_reps.append(reps_batch_5L.cpu())
    sents_reps = torch.cat(sents_reps)

    # 멀티 레이블 데이터를 처리하기 위한 레이블 로직 수정
    labels = torch.tensor(labels, dtype=torch.float32)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/YonseiAI/llama3_2/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    cuda_no = 0  # GPU 번호
    task = 'stackoverflow'  # Task 이름
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 레이블에 해당하는 컬럼 이름 목록
    label_cols = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

    # 데이터셋이 Pandas DataFrame이라고 가정하고, 'Body' 컬럼과 레이블 컬럼들을 추출
    train_df = pd.DataFrame(sof_dataset['train'])
    test_df = pd.DataFrame(sof_dataset['test'])

    # 텍스트 데이터 추출
    train_sents = train_df['Body'].tolist()
    test_sents = test_df['Body'].tolist()

    # 레이블 데이터 추출 (NumPy 배열로 변환)
    train_labels = train_df[label_cols].values.tolist()
    test_labels = test_df[label_cols].values.tolist()

    # train 및 test 데이터를 파일로 저장
    rep_extract(task, 'train', device, train_sents, train_labels, max_len=1024, step=5)
    rep_extract(task, 'test', device, test_sents, test_labels,  max_len=1024, step=5)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

100%|██████████| 4000/4000 [14:49<00:00,  4.50it/s]


torch.Size([20000, 5, 3072])
torch.Size([20000, 9])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 1300/1300 [04:48<00:00,  4.51it/s]


torch.Size([6500, 5, 3072])
torch.Size([6500, 9])


# **2. BERT Embedding**

In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
from transformers import BertTokenizer, BertModel
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels):
    model_path = 'google-bert/bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 10
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            reps_batch = batch_outputs.pooler_output
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    # 멀티 레이블 데이터를 처리하기 위한 레이블 로직 수정
    labels = torch.tensor(labels, dtype=torch.float32)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/YonseiAI/bert/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':

    cuda_no = 0 # GPU 번호
    task = 'stackoverflow' # Task 이름
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 레이블에 해당하는 컬럼 이름 목록
    label_cols = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

    # 데이터셋이 Pandas DataFrame이라고 가정하고, 'Body' 컬럼과 레이블 컬럼들을 추출
    train_df = pd.DataFrame(sof_dataset['train'])
    test_df = pd.DataFrame(sof_dataset['test'])

    # 텍스트 데이터 추출
    train_sents = train_df['Body'].tolist()
    test_sents = test_df['Body'].tolist()

    # 레이블 데이터 추출 (NumPy 배열로 변환)
    train_labels = train_df[label_cols].values.tolist()
    test_labels = test_df[label_cols].values.tolist()

    rep_extract(task, 'train', device, train_sents, train_labels)
    rep_extract(task, 'test', device, test_sents, test_labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 2000/2000 [08:06<00:00,  4.11it/s]


torch.Size([20000, 1024])
torch.Size([20000, 9])


100%|██████████| 650/650 [02:36<00:00,  4.16it/s]


torch.Size([6500, 1024])
torch.Size([6500, 9])


# **3. Roberta Embedding**

In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
from transformers import RobertaTokenizer, RobertaModel
from tqdm import trange
from datasets import load_dataset

def rep_extract(task, mode, device, sents, labels):
    model_path = 'roberta-large'
    tokenizer = RobertaTokenizer.from_pretrained(model_path)
    model = RobertaModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 10  # 작은 배치 크기로 설정하는 것이 좋습니다, 메모리 오류를 방지하기 위해
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            # 첫 번째 토큰([CLS] 토큰에 해당)의 표현을 추출
            reps_batch = batch_outputs.last_hidden_state[:, 0, :]
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    # 멀티 레이블 데이터를 처리하기 위한 레이블 로직 수정
    labels = torch.tensor(labels, dtype=torch.float32)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/YonseiAI/roberta/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    cuda_no = 0 # GPU 번호
    task = 'stackoverflow' # Task 이름

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 레이블에 해당하는 컬럼 이름 목록
    label_cols = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

    # 데이터셋이 Pandas DataFrame이라고 가정하고, 'Body' 컬럼과 레이블 컬럼들을 추출
    train_df = pd.DataFrame(sof_dataset['train'])
    test_df = pd.DataFrame(sof_dataset['test'])

    # 텍스트 데이터 추출
    train_sents = train_df['Body'].tolist()
    test_sents = test_df['Body'].tolist()

    # 레이블 데이터 추출 (NumPy 배열로 변환)
    train_labels = train_df[label_cols].values.tolist()
    test_labels = test_df[label_cols].values.tolist()

    rep_extract(task, 'train', device, train_sents, train_labels)
    rep_extract(task, 'test', device, test_sents, test_labels)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2000/2000 [07:20<00:00,  4.54it/s]


torch.Size([20000, 1024])
torch.Size([20000, 9])


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 650/650 [02:23<00:00,  4.54it/s]


torch.Size([6500, 1024])
torch.Size([6500, 9])


# **4. Dataset 정의**

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

    # 4개의 텐서 파일을 가져옴
    def __init__(self, mode, l_path, b_path, r_path):
        self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
        self.b_sents_reps = torch.load(b_path + f'{mode}_sents.pt')
        self.r_sents_reps = torch.load(r_path + f'{mode}_sents.pt')

        self.labels = torch.load(l_path + f'{mode}_labels.pt')

        self.sample_num = self.labels.shape[0]

    # 인덱스를 받아 해당 샘플의 데이터를 반환하는 메소드
    def __getitem__(self, index):
        return self.l_sents_reps[index], self.b_sents_reps[index], self.r_sents_reps[index], self.labels[index]

    # 데이터셋의 총 샘플 수를 반환하는 메소드
    def __len__(self):
        return self.sample_num

# * Model Operation - 검증셋 테스트

In [2]:

import wandb

# wandb 초기화 (API 키가 환경 변수에 설정되어 있어야 함)
# wandb.init(project="my_project", entity="my_entity", name="my_experiment")
wandb.init(project="Multi output classification", entity ="ai_expert", name = "llm_embed_llama3_2_3B") # Uncomment to use wandb



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import numpy as np
import wandb
import evaluate
from tqdm import tqdm
import torch


# wandb.init(project="huggingface") # Uncomment to use wandb


from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(predictions, references):
    # predictions와 references가 PyTorch 텐서라고 가정하고 있습니다.
    # 시그모이드 함수 적용 (PyTorch 텐서에 대한 연산)
    predictions = torch.sigmoid(predictions)
    # 0.5 임계값 적용 및 데이터 타입 변환 (PyTorch 텐서에 대한 연산)
    predictions = (predictions > 0.5).int()

    # 텐서를 CPU로 이동시키고 NumPy 배열로 변환
    predictions = predictions.cpu().numpy()
    references = references.cpu().numpy()

    # 평가 메트릭 계산
    accuracy = accuracy_score(references, predictions)
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(references, predictions, average='micro')
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(references, predictions, average='macro')
    weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(references, predictions, average='weighted')

    # 평탄화된 정확도 계산
    flat_predictions = predictions.flatten()
    flat_references = references.flatten()
    flat_accuracy = accuracy_score(flat_references, flat_predictions)

    # 평가 메트릭 반환
    return {
        'flat_accuracy': flat_accuracy,
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1,
        'weighted_precision': weighted_precision,
        'weighted_recall': weighted_recall,
        'weighted_f1': weighted_f1,
    }

# Training and validation function for multi-label classification
def Train_and_Evaluate(dataloader_train, dataloader_val, device, model, loss_fn, optimizer):
    # Training
    model.train()
    total_train_loss = 0
    for batch in tqdm(dataloader_train):
        batch_l, batch_b, batch_r, batch_y = [item.to(device) for item in batch]
        pred = model(batch_l.float(), batch_b.float(), batch_r.float())
        loss = loss_fn(pred, batch_y.float())  # Make sure batch_y is a float tensor
        total_train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(dataloader_train)

    # Validation
    model.eval()
    total_val_loss = 0
    all_predictions = []
    all_references = []
    with torch.no_grad():
        for batch in tqdm(dataloader_val):
            batch_l, batch_b, batch_r, batch_y = [item.to(device) for item in batch]
            pred = model(batch_l.float(), batch_b.float(), batch_r.float())
            loss = loss_fn(pred, batch_y.float())  # Make sure batch_y is a float tensor
            total_val_loss += loss.item()

            # Threshold predictions for multi-label classification
            pred_y = (torch.sigmoid(pred) > 0.5).int()
            all_predictions.append(pred_y)
            all_references.append(batch_y)

    # Concatenate all predictions and references
    all_predictions = torch.cat(all_predictions, dim=0)
    all_references = torch.cat(all_references, dim=0)

    avg_val_loss = total_val_loss / len(dataloader_val)

    # Compute metrics
    metrics = compute_metrics(all_predictions, all_references)
    metrics = {'eval/' + key: value for key, value in metrics.items()}
    metrics['train/loss'] = avg_train_loss
    metrics['eval/loss'] = avg_val_loss

    if epoch != 0 and epoch % 40 == 0:
        adjusted_epoch = epoch // 40  # 'epoch'을 2로 나눈 정수 값
        metrics['train/epoch'] = adjusted_epoch
        wandb.log(metrics, step=adjusted_epoch)

    print({metric: f"{value:.4f}" for metric, value in metrics.items()})

# Example usage
# Train_and_Evaluate(dataloader_train, dataloader_val, device, model, loss_fn, optimizer)

# **6. Downstream Model Class**

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DownstreamModel(nn.Module):
    def __init__(self, class_num, SIGMA):
        super(DownstreamModel, self).__init__()
        self.SIGMA = SIGMA
        self.compress_layers = nn.ModuleList()
        for _ in range(5):
            layers = []
            # Changed the input dimension of the first linear layer to 3072
            layers.append(nn.Linear(3072, 1024))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.5))
            self.compress_layers.append(nn.Sequential(*layers))


        # fc1의 입력 차원을 3121로 변경
        self.fc1 = nn.Linear(3121, 1024)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, class_num)
        # 변경된 부분: softmax 대신 sigmoid 활성화 함수 사용
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, input_l, input_b, input_r):
        batch_size = input_l.shape[0]

        # input_l 텐서를 첫 번째 차원을 기준으로 1 크기의 텐서로 분할
        split_tensors = torch.split(input_l, 1, dim=1)
        input = []

        # 분할된 텐서들을 순회
        for i, split_tensor in enumerate(split_tensors):
            # 각 split_tensor를 배치 크기에 맞게 2차원으로 재구성
            split_tensor = split_tensor.reshape(batch_size,-1)
            # 재구성된 텐서를 압축(compress) layer를 거쳐 변환
            input.append(self.compress_layers[i](split_tensor))

        # input_b(bert 임베딩)와 input_r(Roberta 임베딩)을 input에 추가
        input.append(input_b)
        input.append(input_r)
        input = torch.stack(input, dim=1)
        # X * X^T
        input_T = input.transpose(1, 2)
        input_P = torch.matmul(input, input_T)
        input_P = input_P.reshape(batch_size, -1)
        # PN func
        input_P = 2*F.sigmoid(self.SIGMA * input_P) - 1

        a = torch.mean(input_l, dim=1)
        input = torch.cat([input_P, a], dim=1)

        output = self.fc1(input)
        output = self.relu1(output)
        output = self.dropout1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
        output = self.fc3(output)

        # 변경된 부분: 각 레이블의 독립적인 확률을 출력하기 위해 sigmoid 함수를 적용
        # output = torch.sigmoid(output)

        return output


# **7. Fine-tuning**

In [12]:
# 필요한 모듈 import 구문 추가
# from DownstreamModel import DownstreamModel
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
# from model_op import Train, Test
# from model_op_multi import Train_multi, Test_multi
import argparse
import os
import torch
# from MyDataset import MyDataset
import json

if __name__ == '__main__':

    wandb.init(project="Multi output classification", entity="ai_expert", name="llm_embed_llama3_2")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    task = 'stackoverflow'  # 사용 가능한 옵션: 'sst2', 'mr', 'agnews', 'r8', 'r52', 'stackoverflow' 중 하나
    epochs = 201  # 원하는 에폭 수
    SIGMA = 0.1  # SIGMA 값을 설정
    batch_size = 1024  # 배치 크기 설정
    lr = 1e-4  # 학습률 설정

    class_num = {'sst2': 2, 'mr': 2, 'agnews': 5, 'r8': 8, 'r52': 52, 'stackoverflow': 9}
    class_num = class_num[task]

    l_dataset_path = f'/content/drive/MyDrive/YonseiAI/llama3_2/{task}/dataset_tensor/'
    b_dataset_path = f'/content/drive/MyDrive/YonseiAI/bert/{task}/dataset_tensor/'
    r_dataset_path = f'/content/drive/MyDrive/YonseiAI/roberta/{task}/dataset_tensor/'

    # Train dataset
    train_data = MyDataset('train', l_dataset_path, b_dataset_path, r_dataset_path)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    # Test dataset
    test_data = MyDataset('test', l_dataset_path, b_dataset_path, r_dataset_path)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    model = DownstreamModel(class_num, SIGMA).to(device)

    loss_fn = nn.BCEWithLogitsLoss().to(device)  # 멀티 레이블 손실 함수
    optimizer = optim.Adam(model.parameters(), lr=lr)

    print('training ...')
    for epoch in range(epochs):
        model = model.to(device)
        print(f'--------------------------- epoch {epoch} ---------------------------')
        Train_and_Evaluate(train_loader, test_loader, device, model, loss_fn, optimizer)

    # 모델 학습 후, 모델의 가중치 저장하기
    model_save_path = f"/content/drive/MyDrive/YonseiAI/{task}_model_weights_llama3_2_3b.pth"
    torch.save(model.state_dict(), model_save_path)
    wandb.finish()

<ipython-input-1-1ab9f002cd4f>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
<ipython-input-1-1ab9f002cd4f>:9:

training ...
--------------------------- epoch 0 ---------------------------


100%|██████████| 7/7 [00:00<00:00, 14.34it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval/flat_accuracy': '0.8651', 'eval/accuracy': '0.0000', 'eval/macro_precision': '0.0000', 'eval/macro_recall': '0.0000', 'eval/macro_f1': '0.0000', 'eval/micro_precision': '0.0000', 'eval/micro_recall': '0.0000', 'eval/micro_f1': '0.0000', 'eval/weighted_precision': '0.0000', 'eval/weighted_recall': '0.0000', 'eval/weighted_f1': '0.0000', 'train/loss': '0.5293', 'eval/loss': '0.3969'}
--------------------------- epoch 1 ---------------------------


100%|██████████| 7/7 [00:00<00:00, 15.22it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

100%|██████████| 7/7 [00:00<00:00, 13.62it/s]


{'eval/flat_accuracy': '0.9510', 'eval/accuracy': '0.6560', 'eval/macro_precision': '0.7045', 'eval/macro_recall': '0.8511', 'eval/macro_f1': '0.7559', 'eval/micro_precision': '0.7980', 'eval/micro_recall': '0.8532', 'eval/micro_f1': '0.8247', 'eval/weighted_precision': '0.8112', 'eval/weighted_recall': '0.8532', 'eval/weighted_f1': '0.8294', 'train/loss': '0.0948', 'eval/loss': '0.1378'}
--------------------------- epoch 200 ---------------------------


100%|██████████| 7/7 [00:00<00:00, 14.50it/s]


{'eval/flat_accuracy': '0.9524', 'eval/accuracy': '0.6638', 'eval/macro_precision': '0.7139', 'eval/macro_recall': '0.8402', 'eval/macro_f1': '0.7586', 'eval/micro_precision': '0.8084', 'eval/micro_recall': '0.8479', 'eval/micro_f1': '0.8277', 'eval/weighted_precision': '0.8178', 'eval/weighted_recall': '0.8479', 'eval/weighted_f1': '0.8311', 'train/loss': '0.0946', 'eval/loss': '0.1341', 'train/epoch': '5.0000'}


eval/accuracy,▁▅█▆█
eval/flat_accuracy,▁▅█▆█
eval/loss,█▂▁▂▂
eval/macro_f1,▁▅███
eval/macro_precision,▁██▅▆
eval/macro_recall,▁▄▇█▇
eval/micro_f1,▁▆█▇█
eval/micro_precision,▆▆█▁▅
eval/micro_recall,▁▅▆██
eval/weighted_f1,▁▆█▇█
eval/weighted_precision,▇▇█▁▄


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel, AutoConfig

# Llama, BERT, RoBERTa 토크나이저 및 모델 로드
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B", trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token  # 패딩 토큰 설정
llama_config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-3B", trust_remote_code=True)
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", config=llama_config, trust_remote_code=True)

bert_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
bert_model = BertModel.from_pretrained('bert-large-uncased')

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
roberta_model = RobertaModel.from_pretrained('roberta-large')

# 모든 모델을 평가 모드로 설정하고 장치로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llama_model.eval().to(device)
bert_model.eval().to(device)
roberta_model.eval().to(device)

# 다운스트림 모델 초기화 및 가중치 로드
class_num = 9  # 예시 클래스 수
SIGMA = 0.1  # SIGMA 값
downstream_model = DownstreamModel(class_num, SIGMA).to(device)
model_load_path = "/content/drive/MyDrive/YonseiAI/stackoverflow_model_weights_llama3_2_3b.pth"
downstream_model.load_state_dict(torch.load(model_load_path, map_location=device))
downstream_model.eval()

# Llama 임베딩 추출 함수
def get_llama_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        # 마지막 5개의 레이어 평균
        embedding = torch.stack([torch.mean(outputs.hidden_states[i], dim=1) for i in range(-1, -6, -1)], dim=1)
    return embedding

# BERT 임베딩 추출 함수
def get_bert_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.pooler_output  # BERT 임베딩
    return embedding

# RoBERTa 임베딩 추출 함수
def get_roberta_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]  # 첫 번째 토큰 ([CLS] 토큰)
    return embedding

# 추론 함수
def infer(text, downstream_model, device):
    # 각 모델로부터 임베딩 추출
    llama_emb = get_llama_embedding(text, llama_tokenizer, llama_model, device)
    bert_emb = get_bert_embedding(text, bert_tokenizer, bert_model, device)
    roberta_emb = get_roberta_embedding(text, roberta_tokenizer, roberta_model, device)

    # 모든 임베딩을 float 타입으로 변환
    llama_emb = llama_emb.float()
    bert_emb = bert_emb.float()
    roberta_emb = roberta_emb.float()

    # 다운스트림 모델을 통한 예측 수행
    with torch.no_grad():
        prediction = downstream_model(llama_emb, bert_emb, roberta_emb)
        prediction = torch.sigmoid(prediction)  # 확률로 변환

    return prediction


model-00001-of-00002.safetensors:  10%|9         | 493M/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-17-d9b95250128d>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We reco

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [18]:

# 예측된 클래스를 레이블로 맵핑하여 출력하는 함수
def print_predicted_labels(predicted_classes, labels):
    # 예측된 클래스 중 1인 레이블만 선택
    predicted_labels = [labels[i] for i, pred in enumerate(predicted_classes) if pred == 1]

    # 선택된 레이블 출력
    print("Predicted labels:", predicted_labels)


# 레이블 목록
labels = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "working of compareTo() method of Comparable interface            I have one Employee class and the requirement is to sort the objects using comparable interface. The output with this code is :The difference of this id and other id is..** 6  other id**1The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**1The difference of this id and other id is..** 11  other id**3The difference of this id and other id is..** 11  other id**6",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = infer(text, downstream_model, device)

    # 예측 결과를 CPU로 이동
    prediction = prediction.cpu()
    predicted_classes = (prediction > 0.5).int().numpy()

    # 예측 결과 출력
    print("Predicted probabilities:", prediction.numpy())
    print("Predicted classes:", predicted_classes)


    print_predicted_labels(predicted_classes[0], labels)

Predicted probabilities: [[1.0000000e+00 1.2594886e-14 5.3181827e-01 9.0126347e-15 1.3499197e-11
  5.2472944e-12 4.6629893e-13 6.4584224e-06 2.4949493e-15]]
Predicted classes: [[1 0 1 0 0 0 0 0 0]]
Predicted labels: ['Algorithms', 'Data Science']
